In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [29]:
df = pd.read_csv('challenge_immo_eliza_analysis/data/20220208_Final_result.csv')


In [30]:
### Remove Null Price rows and change type to float
df = df[pd.to_numeric(df['Price'], errors='coerce').notnull()]
df = df.astype({"Price": float}, errors='raise')


In [31]:
df['Living area'].isnull().sum()

3071

In [32]:
### Remove Null Living Area
### Living area
df = df[pd.to_numeric(df['Living area'], errors='coerce').notnull()]

In [33]:
df['Living area'].isnull().sum()

0

In [34]:
### Remove duplicate ID
df = df.drop_duplicates(['Immoweb ID'], keep='last')


In [35]:
### Remove subtype 
df = df[df['property sub-type'] != "APARTMENT_BLOCK"]


In [36]:
### Building condition
building_condition_map = {'As new': 6, 'Just renovated': 5, 'Good': 4, 'To be done up': 3, 'To renovate':2, 'To restore':1}
df = df.applymap(lambda s: building_condition_map.get(s) if s in building_condition_map else s)

df['Building condition'] = df['Building condition'].fillna(2)

In [37]:
### One hot encoding for property sub-type Property type & Post code
df['Prop_orig'] = df['Property type']
df['Post_code_orig'] = df['Post code']
df["Sub-type"] = df["property sub-type"]
df = pd.get_dummies(df, columns=['Property type', 'Post code', "property sub-type"])


In [38]:
### Kitchen type
Kit_type_dict = {"USA uninstalled" : 0, 
                 "Not installed" : 0, 
                 "Installed": 1, 
                 "USA installed": 1,
                 "Semi equipped": 1,
                 "USA semi equipped": 1,
                 "Hyper equipped": 2,
                 "USA hyper equipped": 2
                }

df = df.replace(Kit_type_dict)
df["Kitchen type"] = df["Kitchen type"].fillna(0)

In [39]:
### Furnished (fill No to missing value then map to 1,0)
df['Furnished'] = df['Furnished'].fillna("No")
df['Furnished'] = df['Furnished'].apply(lambda v: 0 if v == "No" else 1)

In [40]:
### Bedrooms (Fill missing values with 2 bedrooms)
df['Bedrooms'] = df['Bedrooms'].fillna(2).astype(int)


In [41]:
### Terrace (Combine terrace & terrace surface in yes and no)

df['Terrace_Combined'] = np.where(df['Terrace surface'].isnull() & df['Terrace'].isnull(), 'No', 'Yes')
terrace = ['No', 'Yes']
ordenc = OrdinalEncoder(categories=[terrace])
df[["Terrace_Combined"]] = ordenc.fit_transform(df[["Terrace_Combined"]])

df = df.drop(columns =['Terrace', 'Terrace surface'])

In [42]:
### Garden (Combine ['Garden'] and ['Garden surface] as ['garden_label] and map to 1,0)
df.loc[df['Garden surface'].isna()]
def categorise(col):  
    if col['Garden'] == "Yes" or col['Garden surface'] > 0:
        return 1
    return 0


df['garden_label'] = df.apply(lambda col: categorise(col), axis=1)
df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,How many fireplaces?,...,property sub-type_MANOR_HOUSE,property sub-type_MANSION,property sub-type_MIXED_USE_BUILDING,property sub-type_PENTHOUSE,property sub-type_SERVICE_FLAT,property sub-type_TOWN_HOUSE,property sub-type_TRIPLEX,property sub-type_VILLA,Terrace_Combined,garden_label
0,9729720,179000.0,4.0,1.0,1,1,No,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1.0,0
1,9729785,255000.0,4.0,1.0,1,1,No,2.0,NaN,NaN,...,0,0,0,1,0,0,0,0,1.0,0
2,9729784,255000.0,4.0,1.0,1,1,No,2.0,NaN,NaN,...,0,0,0,1,0,0,0,0,1.0,0
3,9729780,620000.0,4.0,1.0,3,1,No,2.0,NaN,NaN,...,0,0,0,1,0,0,0,0,1.0,0
4,9727201,379000.0,4.0,2.0,2,0,No,3.0,No,NaN,...,0,0,0,1,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14531,8041977,217000.0,6.0,0.0,1,0,No,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,1.0,0
14532,9459825,148500.0,2.0,0.0,1,0,No,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0.0,0
14533,9581654,253515.0,2.0,0.0,1,0,No,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,1.0,0
14539,7921693,1350000.0,2.0,0.0,30,0,No,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0.0,0


In [43]:
### Drop tenement building, Garden orientation, Fireplace
df = df.drop(columns =['Tenement building'])

In [44]:
df = df.drop(columns =['Garden orientation'])

In [45]:
df = df.drop(columns =["How many fireplaces?"])

In [46]:
### Swimming pool (Fill missing values with value 0)
df['Swimming pool'].fillna(0, inplace = True)
df['Swimming pool'] = df['Swimming pool'].apply(lambda v: 0 if v == "No" else 1)

In [47]:
### Surface of the plot
# Fill empty values with 0
df['Surface of the plot'].fillna(0, inplace = True)

In [48]:
### Frontages
#get ['number of frontages'] with values and calc mean
selected_rows = df[~df['Number of frontages'].isnull()]
mean_num_of_frontages = selected_rows['Number of frontages'].mean(axis=0).round(0)
# fill mean value to missing value
df['Number of frontages'] = df['Number of frontages'].fillna(mean_num_of_frontages)

In [49]:
###
df['price/m2'] = (df['Price']/ df['Living area']).round(2)

In [51]:
df.to_csv('data_15feb.csv')